## MDPs in pymdptoolbox - Class Assignment

In this practical exercise, we will look at how MDP planning is implemented in a mathematical toolkit, and track the calculation of the rewards for each state via Value Iteration. The following code sets up an MDP environment (the basic case shown in class, shown in the Figure below) and computes the policy for the given MDP using the Value Iteration algorithm.

<img align="center" src="mdp_simple.png"/>

Then we provide a set of questions for you to implement and answer. This assignment is not graded.



In [25]:
import sys
sys.path.insert(1,'pymdptoolbox/src')
import mdptoolbox.example

import numpy as _np
from gen_scenario import *
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [26]:
# The line below is to be used if you have pymdptoolbox installed with setuptools
# import mdptoolbox.example
# Whereas the line below obviate the need to install that

"""
(Y,X)
| 00 01 02 ... 0X-1       'N' = North
| 10  .         .         'S' = South
| 20    .       .         'W' = West
| .       .     .         'E' = East
| .         .   .         'T' = Terminal
| .           . .         'O' = Obstacle
| Y-1,0 . . .   Y-1X-1
""" 

shape = [3,4]
rewards = [[0,3,100],[1,3,-100]]
obstacles = [[1,1]]
terminals = [[0,3],[1,3]]
P, RSS, R = mdp_grid(shape=shape, terminals=terminals, r=-3, rewards=rewards, obstacles=obstacles)
vi = mdptoolbox.mdp.ValueIterationGS(P, R, discount=0.5, epsilon=0.001, max_iter=1000, skip_check=True)
vi.verbose = True
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)
display_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)


 Iteration   Variation
         1  200.000000
         2   41.260000
         3   16.483000
         4    6.465697
         5    1.551601
         6    0.295707
         7    0.060178
         8    0.015785
         9    0.005540
        10    0.002253
        11    0.000981
Iterating stopped, epsilon-optimal policy found.
[['E' 'E' 'E' 'T']
 ['N' 'O' 'N' 'T']
 ['N' 'E' 'N' 'S']]


→,→,→,◎
↑,◾,↑,◎
↑,→,↑,↓


In [27]:
def plot(P, R, discounts=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95], epsilon=0.001, max_iter=1000):
    data_list = []
    for d in discounts:
        vis = mdptoolbox.mdp.ValueIteration(P, R, d, epsilon, max_iter, skip_check=True)
        vis.run()
        iterations = 1
        for value in vis.iterations_list:
            data_list.append([value, 'gamma: ' + str(d), iterations])
            iterations +=1
    data_frame2 = pd.DataFrame(data_list, columns=['difference', 'discount', 'iterations'])
    ax = sns.lineplot(x = 'iterations', y = 'difference', hue='discount',  data=data_frame2)
plot(P,R)

AttributeError: module 'seaborn' has no attribute 'lineplot'

You can use the following commands to check the difference of value-function values during each teration and the values of each value function.


In [28]:
print(vi.iterations_list)
print(vi.v_list[-1])

[200.0, 41.260000000000005, 16.483000000000004, 6.4656972263671886, 1.5516013086428719, 0.29570678878162315, 0.060177953646783244, 0.015784916353677847, 0.0055399796787702904, 0.0022526739209762781, 0.00098114810443483691]
[   2.69494788   14.16745349   39.37677054  100.           -2.13558073
   -3.            8.15864023 -100.           -4.23674139   -3.41357286
   -0.18068356   -5.47018326]


Before you go for the questionnaire, take your time to open the source code of the MDP toolkit we use, specifically, look into these files:
1. [gen_scenario.py](gen_scenario.py) - contains the conversion code to make the simple coordinate commands above (e.g. ```shape = [3,4]```) into the matrices actually used by the MDP solver
2. [mdp.py](pymdptoolbox/src/mdptoolbox/mdp.py) - contains most of the logic for an MDP, including the *Bellman Equation* as follows:

$$V(s) = \left[ \max_{a} \gamma \sum_{s'}P(s'|s,a)*V(s') \right]+ R(s)$$

See if you can identify how this equation is implemented in the ```MDP._bellmanOperator``` with the [mdp.py](pymdptoolbox/src/mdptoolbox/mdp.py) file. Note how this implementation uses matrix multiplication to achieve the summation step described in the equation. Once you believe you understand that, go ahead and respond the questionnaire. 

### Questionnaire
1. Study the code of the cell above and answer the following questions.
	1. What is the policy generated if we change the discount factor of the grid domain to ```0.1```?
	2. Use the following line ```vi.verbose = True``` before ```vi.run()```:   
	What is the variation for each of the first three iterations with the discount factor of ```0.9``` and how many iterations does the algorithm take to converge?
	3. How does changes to the discount factor affect the variation of the state values over time?

In [29]:
# 1.A
# [['E' 'E' 'E' 'T']
#  ['N' 'O' 'W' 'T']
#  ['N' 'E' 'N' 'S']]

In [30]:
# 1.B
# Iteration   Variation
#          1  200.000000
#          2   74.354400
#          3   53.412696
#             ...
#         16    0.000057
# Iterating stopped, epsilon-optimal policy found.

In [31]:
# 1.C
# It not only affect the pecentage of discount over the variation at each iteration, it also
# affects the number of iterations to reach optimal result because the more it's closer to 1
# the more it ignores future rewards over each iteration.
# For discount = 0.1 the optimal result is at the 5th iteration.
# Iteration   Variation
#          1  200.000000
#          2    8.242400
#          3    0.659224
#          4    0.052638
#          5    0.002211
# If it increases to 0.5 the optimal result is only reached ath the 11th.
# Iteration   Variation
#         1  200.000000
#         2   41.260000
#         3   16.483000
#          ...
#        10    0.002253
#        11    0.000981

2. The scenario below has an interesting structure whereby the positive rewarding terminal state is partially surrounded by negatively-rewarding states. Program this scenario in pymdptoolbox and compute the optimal policy with a discount factor of 0.99.

<img align="center" src="mdp-odd.png"/>




In [32]:
#2
shape = [5,5]
rewards = [[2,1,-100],[1,2,-100],[2,2,100],[3,2,-100]]
obstacles = [[1,1],[1,4],[3,1],[3,3]]
terminals = [[2,1],[1,2],[2,2],[3,2]]
P, RSS, R = mdp_grid(shape=shape, terminals=terminals, r=-3, rewards=rewards, obstacles=obstacles)
vi = mdptoolbox.mdp.ValueIterationGS(P, R, discount=0.99, epsilon=0.001, max_iter=1000, skip_check=True)
vi.verbose = True
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)
display_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)


 Iteration   Variation
         1  200.000000
         2   53.426516
         3   41.785472
         4   24.594519
         5   20.056584
         6   17.358982
         7   14.044100
         8   13.179184
         9   10.020322
        10    6.183187
        11    5.195060
        12    3.163524
        13    1.621558
        14    0.764909
        15    0.341000
        16    0.145604
        17    0.060068
        18    0.024092
        19    0.009439
        20    0.003626
        21    0.001370
        22    0.000510
        23    0.000187
        24    0.000068
        25    0.000025
        26    0.000009
Iterating stopped, epsilon-optimal policy found.
[['E' 'E' 'E' 'S' 'W']
 ['N' 'O' 'T' 'S' 'O']
 ['S' 'T' 'T' 'W' 'W']
 ['S' 'O' 'T' 'O' 'N']
 ['E' 'E' 'E' 'E' 'N']]


→,→,→,↓,←
↑,◾,◎,↓,◾
↓,◎,◎,←,←
↓,◾,◎,◾,↑
→,→,→,→,↑


3. Define two new 5 by 5 scenarios with multiple obstacles and an interesting geometry following the guidelines below. Calculate the policy with discount factor 0.99, and then try to explain intuitively the reason for the resulting policies, given the initial parameters. These two scenarios must have the following characteristics:
	1. A scenario with one (or more) terminal states with positive rewards and at least one other state with the same amount of, but negative reward and no terminal states with negative rewards.
	2. A scenario with one terminal state with a negative reward and at least one non-terminal state with a positive reward.

In [33]:
#### 3.A
shape = [5, 5]
rewards = [[2, 3, 100], [2, 1, -100], [4, 1, -100], [4, 4, 100]]
obstacles = [[1, 3], [3, 3], [3, 4]]
terminals = [[2, 3], [4, 4]]
P, RSS, R = mdp_grid(shape=shape, terminals=terminals, r=-3, rewards=rewards, obstacles=obstacles)
vi = mdptoolbox.mdp.ValueIterationGS(P, R, discount=0.99, epsilon=0.001, max_iter=1000, skip_check=True)
# vi.verbose = True
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)
display_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)

[['E' 'E' 'S' 'E' 'S']
 ['N' 'N' 'S' 'O' 'S']
 ['N' 'E' 'E' 'T' 'W']
 ['E' 'E' 'N' 'O' 'O']
 ['N' 'E' 'E' 'E' 'T']]


→,→,↓,→,↓
↑,↑,↓,◾,↓
↑,→,→,◎,←
→,→,↑,◾,◾
↑,→,→,→,◎


In [34]:
#3.B
shape = [5, 5]
rewards = [[2, 3, 100], [4, 1, -100], [4, 4, 100]]
obstacles = [[2, 0], [2, 1], [1, 3], [3, 3], [3, 4]]
terminals = [[4, 1]]
P, RSS, R = mdp_grid(shape=shape, terminals=terminals, r=-3, rewards=rewards, obstacles=obstacles)
vi = mdptoolbox.mdp.ValueIterationGS(P, R, discount=0.99, epsilon=0.001, max_iter=1000, skip_check=True)
#vi.verbose = True
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)
display_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)

[['E' 'E' 'S' 'W' 'S']
 ['E' 'E' 'S' 'O' 'S']
 ['O' 'O' 'S' 'W' 'W']
 ['E' 'N' 'S' 'O' 'O']
 ['W' 'T' 'E' 'E' 'E']]


→,→,↓,←,↓
→,→,↓,◾,↓
◾,◾,↓,←,←
→,↑,↓,◾,◾
←,◎,→,→,→
